<a href="https://colab.research.google.com/github/JunmoJeong/credit_risk_modeling/blob/main/credit_risk_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
채무 불이행 예측
문제를 정의하고 사용 사례를 이해하고 나면 개발하기 쉽다.
금융기관은 채무자가 미래에 대출을 갚기에 충분한 돈을 지니게 될지 여부를 조사한다. 
분석 ouput은 확률이다. (채무자의 채무 불이행 가능성)
여기서 확률은 고객이 제때 대출에 대한 월별 상환액(EMI)을 지불하지 않을 가능성을 나타낸다.

높은 확률 값은 고객이 금융기관의 채무 불이행자 또는 부적합 채무자가 될 가능성을 나타낸다. 
'''

In [ ]:
# 데이터 특성
